In [12]:
import jsonpickle
from collections import defaultdict
import pandas as pd

In [6]:
target_media = (
    # ABC assume national reach. 230 articles.
    ('ABC Premium News; Sydney', 'neutral', 'national'),
    ('7.30; Sydney', 'neutral', 'national'),

    # ACT. 951 articles. But Australian prolly national reach.
    # ('The Australian (Online); Canberra, A.C.T.', 'right', 'national'),
    ('The Canberra Times; Canberra, A.C.T.', 'left', 'ACT'),
    
    # NSW
    ('News.com.au; Sydney, N.S.W.', 'right', 'national'),
    ('Sydney Morning Herald; Sydney, N.S.W.', 'left', 'NSW'),
    ('Sun-Herald; Sydney, N.S.W.', 'left', 'NSW'),
    ('The Daily Telegraph (Online); Surrey Hills, N.S.W.', 'right', 'NSW'),

    # VIC
    ('The Age; Melbourne, Vic.', 'left', 'VIC'),
    ('Herald Sun; Melbourne, Vic.', 'left', 'VIC'),
    ('Sunday Age; Melbourne, Vic.', 'left', 'VIC'),

    # SA
    ('The Advertiser; Adelaide, S. Aust.', 'right', 'SA'),
    
    # QLD
    ('The Courier - Mail; Brisbane, Qld.', 'right', 'QLD'),
    ('The Cairns Post; Cairns, Qld.', 'right', 'QLD'),
    
    # NT
    ('The Northern Territory News; Darwin, N.T.', 'unclear', 'NT'),

    # TAS
    ('Advocate; Burnie, Tas.', 'unclear', 'TAS'),
    ('The Examiner; Launceston, Tas.', 'unclear', 'TAS'),
    ('The Mercury (Online); Hobart Town', 'unclear', 'TAS'),
)

# Quick check handcrafted attribute.
# set(c for a,b,c in target_media)

## Check MFD2 count per media and state

In [39]:
voice_dict = None
with open("./data/pickle_240227/voice_m2bu.pickle") as f:
    voice_dict = jsonpickle.decode(f.read())

pub_title_dict = {media[0]: defaultdict(int) for media in target_media}

for v in voice_dict.values():
    pub_title = v['pub_title']
    if pub_title in pub_title_dict:
        mf_info = v['mfd2_bert_uncased'][3]
        for sent_info in mf_info:
            for word_info in sent_info:
                for word_mf in word_info.values():
                    pub_title_dict[pub_title][word_mf] += 1

In [43]:
states = set([element[2] for element in target_media])
stances = set([element[1] for element in target_media])

stance_count2 = {stance: defaultdict(int) for stance in stances}
state_count2 = {state: defaultdict(int) for state in states}

for k, v in pub_title_dict.items():
    for media_info in target_media:
        if k == media_info[0]:
            stance = media_info[1]
            state =  media_info[2]
            for mf, count in v.items():
                stance_count2[stance][mf] += count
                state_count2[state][mf] += count

## Check MFD count per media and state

In [44]:
voice_dict = None
with open("./data/pickle_240227/voice_mbu.pickle") as f:
    voice_dict = jsonpickle.decode(f.read())

pub_title_dict = {media[0]: defaultdict(int) for media in target_media}

for v in voice_dict.values():
    pub_title = v['pub_title']
    if pub_title in pub_title_dict:
        mf_info = v['mfd_bert_uncased'][3]
        for sent_info in mf_info:
            for word_info in sent_info:
                for word_mf in word_info:
                    pub_title_dict[pub_title][word_mf] += 1

In [45]:
# Aggregate moral foundation count for stance and state
# target_media[i][1] - stance
# target_media[i][2] - state

states = set([element[2] for element in target_media])
stances = set([element[1] for element in target_media])

stance_count = {stance: defaultdict(int) for stance in stances}
state_count = {state: defaultdict(int) for state in states}

for k, v in pub_title_dict.items():
    for media_info in target_media:
        if k == media_info[0]:
            stance = media_info[1]
            state =  media_info[2]
            for mf, count in v.items():
                stance_count[stance][mf] += count
                state_count[state][mf] += count

In [18]:
stance_count

{'left': defaultdict(int,
             {'loyalty.virtue': 2920,
              'authority.virtue': 1872,
              'moral': 862,
              'sanctity.virtue': 130,
              'fairness.vice': 115,
              'care.vice': 491,
              'care.virtue': 315,
              'loyalty.vice': 113,
              'fairness.virtue': 677,
              'authority.vice': 199,
              'sanctity.vice': 43}),
 'right': defaultdict(int,
             {'authority.virtue': 852,
              'loyalty.virtue': 1572,
              'authority.vice': 67,
              'fairness.virtue': 288,
              'care.vice': 279,
              'care.virtue': 204,
              'moral': 376,
              'loyalty.vice': 83,
              'sanctity.virtue': 43,
              'sanctity.vice': 36,
              'fairness.vice': 48}),
 'neutral': defaultdict(int,
             {'fairness.virtue': 330,
              'loyalty.virtue': 1957,
              'moral': 355,
              'care.vice': 221,
 

In [27]:
pd.DataFrame(state_count).T.sort_values(by=['fairness.virtue'], ascending=False)

,fairness.virtue,authority.virtue,moral,care.vice,loyalty.virtue,care.virtue,authority.vice,loyalty.vice,fairness.vice,sanctity.vice,sanctity.virtue
NSW,369,948,412,258,1373,193,100,67,57,35,63
national,330,873,355,221,1957,187,61,58,43,43,29
VIC,277,832,357,204,1181,132,84,48,44,17,53
ACT,201,484,277,149,975,87,49,36,34,16,37
SA,71,234,108,74,588,49,14,15,13,5,12
TAS,51,183,88,44,397,45,9,14,12,9,3
QLD,47,226,84,85,375,58,19,30,15,6,8
NT,38,164,58,35,328,39,10,6,8,6,11


In [28]:
pd.DataFrame(stance_count).T.sort_values(by=['fairness.virtue'], ascending=False)

,loyalty.virtue,authority.virtue,moral,sanctity.virtue,fairness.vice,care.vice,care.virtue,loyalty.vice,fairness.virtue,authority.vice,sanctity.vice
left,2920,1872,862,130,115,491,315,113,677,199,43
neutral,1957,873,355,29,43,221,187,58,330,61,43
right,1572,852,376,43,48,279,204,83,288,67,36
unclear,725,347,146,14,20,79,84,20,89,19,15


## mfd count vs mfd2 count

In [53]:
states = set([element[2] for element in target_media])
stances = set([element[1] for element in target_media])

stance_count_diff = {stance: defaultdict(int) for stance in stances}
state_count_diff = {state: defaultdict(int) for state in states}

for stance, v in stance_count.items():
    for mf, count in v.items():
        stance_count_diff[stance][mf] = stance_count2[stance][mf] - stance_count[stance][mf]

pd.DataFrame(stance_count_diff)

,left,right,neutral,unclear
loyalty.virtue,499,237,305,108
authority.virtue,-59,4,8,-1
moral,-862,-376,-355,-146
sanctity.virtue,635,286,330,118
fairness.vice,292,130,122,53
care.vice,3,8,19,16
care.virtue,505,248,410,102
loyalty.vice,-100,-78,-50,-20
fairness.virtue,366,160,198,108
authority.vice,-112,-23,-24,-6


In [60]:
for state, v in state_count.items():
    for mf, count in v.items():
        state_count_diff[state][mf] = state_count2[state][mf] - state_count[state][mf]

pd.DataFrame(state_count_diff)
pd.DataFrame(state_count2)

,QLD,VIC,NT,NSW,national,SA,ACT,TAS
loyalty.virtue,418,1412,361,1659,2262,649,1090,472
fairness.virtue,81,421,78,531,528,120,338,119
sanctity.virtue,99,299,64,356,359,89,251,68
authority.virtue,222,771,163,955,881,238,483,183
care.virtue,117,331,77,395,597,148,281,109
sanctity.vice,27,62,16,96,81,27,58,22
authority.vice,12,44,8,65,37,2,8,5
care.vice,76,203,51,246,240,91,165,44
fairness.vice,36,147,27,227,165,53,122,46
loyalty.vice,3,5,0,4,8,1,5,0


## Check mfd, mfd2, and emfd

In [29]:
from emfdscore.load_mfds import mfd2, emfd_single_vice_virtue, mfd_regex, mfd

In [61]:
mfd2

{'compassion': {'foundation': 'care.virtue'},
 'empathy': {'foundation': 'care.virtue'},
 'kindness': {'foundation': 'care.virtue'},
 'caring': {'foundation': 'care.virtue'},
 'generosity': {'foundation': 'care.virtue'},
 'benevolence': {'foundation': 'care.virtue'},
 'altruism': {'foundation': 'care.virtue'},
 'compassionate': {'foundation': 'care.virtue'},
 'nurture': {'foundation': 'care.virtue'},
 'gentleness': {'foundation': 'care.virtue'},
 'nurturance': {'foundation': 'care.virtue'},
 'sympathy': {'foundation': 'care.virtue'},
 'nurturing': {'foundation': 'care.virtue'},
 'motherly': {'foundation': 'care.virtue'},
 'love': {'foundation': 'care.virtue'},
 'beneficence': {'foundation': 'care.virtue'},
 'empathize': {'foundation': 'care.virtue'},
 'helpfulness': {'foundation': 'care.virtue'},
 'loving': {'foundation': 'care.virtue'},
 'pity': {'foundation': 'care.virtue'},
 'mercy': {'foundation': 'care.virtue'},
 'nurturer': {'foundation': 'care.virtue'},
 'compassionately': {'fou